<a href="https://colab.research.google.com/github/maxpark/DAC_SDC_2020_Yolov3_model/blob/master/train_yolov3_20201023_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##-- Torch import

import time
import glob
import torch
import os, sys

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.6.0+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15079MB, multi_processor_count=40)


In [2]:
!git clone https://github.com/maxpark/DAC_SDC_2020_Yolov3_model

Cloning into 'DAC_SDC_2020_Yolov3_model'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 86 (delta 37), reused 25 (delta 2), pack-reused 0
Unpacking objects: 100% (86/86), done.


In [ ]:
#!bash train/get_dataset.sh

In [ ]:
#!wget -O data_training_V4.zip https://drive.google.com/file/d/1cwPeRERlwHsOFMOYR2K24XPCsi311iER/view?usp=sharing

--2020-09-20 06:58:44--  https://drive.google.com/file/d/1cwPeRERlwHsOFMOYR2K24XPCsi311iER/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 108.177.97.138, 108.177.97.113, 108.177.97.100, ...
Connecting to drive.google.com (drive.google.com)|108.177.97.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data_training_V4.zip’

data_training_V4.zi     [ <=>                ]  60.97K  --.-KB/s    in 0.1s    

2020-09-20 06:58:45 (508 KB/s) - ‘data_training_V4.zip’ saved [62433]



In [ ]:
#!filename="data_training_V4.zip"
#!fileid="1cwPeRERlwHsOFMOYR2K24XPCsi311iER" 
#!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=${fileid}" >/dev/null
#!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$(awk '/download/ {print $NF}' ./cookie)&id=${fileid}" -o ${filename}


curl: option -o: requires parameter
curl: try 'curl --help' or 'curl --manual' for more information


In [ ]:
#!f="val2017.zip" && curl http://images.cocodataset.org/zips/$f -o $f && unzip -q $f && rm $f

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  777M  100  777M    0     0  12.7M      0  0:01:00  0:01:00 --:--:-- 16.3M


In [3]:
#
from google.colab import drive
drive.mount('/content/drive')
#os.listdir("/content/drive/My Drive")
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)

Mounted at /content/drive


In [ ]:
!cp '/content/drive/My Drive/data_training_V4.zip' .  

In [ ]:
!unzip  data_training_V4.zip

Streaming output truncated to the last 5000 lines.
  inflating: data_training/wakeboard3/000737.xml  
  inflating: data_training/wakeboard3/000738.jpg  
  inflating: data_training/wakeboard3/000738.xml  
  inflating: data_training/wakeboard3/000739.jpg  
  inflating: data_training/wakeboard3/000739.xml  
  inflating: data_training/wakeboard3/000740.jpg  
  inflating: data_training/wakeboard3/000740.xml  
  inflating: data_training/wakeboard3/000741.jpg  
  inflating: data_training/wakeboard3/000741.xml  
  inflating: data_training/wakeboard3/000742.jpg  
  inflating: data_training/wakeboard3/000742.xml  
  inflating: data_training/wakeboard3/000743.jpg  
  inflating: data_training/wakeboard3/000743.xml  
  inflating: data_training/wakeboard3/000744.jpg  
  inflating: data_training/wakeboard3/000744.xml  
  inflating: data_training/wakeboard3/000745.jpg  
  inflating: data_training/wakeboard3/000745.xml  
  inflating: data_training/wakeboard3/000746.jpg  
  inflating: data_training/wake

In [ ]:
#!cd 'DAC_SDC_2020_Yolov3_model/dac_sdc_2020/'
os.chdir('/content/DAC_SDC_2020_Yolov3_model/dac_sdc_2020/')
!pwd


/content/DAC_SDC_2020_Yolov3_model/dac_sdc_2020


In [ ]:
os.chdir('/content')
!python3 /content/DAC_SDC_2020_Yolov3_model/dac_sdc_2020/split_data.py

Splitting DAC SDC 2020 dataset from data_training to dac_sdc_2020_dataset.

The dataset contains 93520 images.

There're 95 classes: ['boat1', 'boat2', 'boat3', 'boat4', 'boat5', 'boat6', 'boat7', 'boat8', 'building1', 'building2', 'building3', 'car1', 'car10', 'car11', 'car12', 'car13', 'car14', 'car15', 'car16', 'car17', 'car18', 'car19', 'car2', 'car20', 'car21', 'car22', 'car23', 'car24', 'car3', 'car4', 'car5', 'car6', 'car8', 'car9', 'drone1', 'drone2', 'drone3', 'drone4', 'group2', 'group3', 'horseride1', 'paraglider1', 'person1', 'person10', 'person11', 'person12', 'person13', 'person14', 'person15', 'person16', 'person17', 'person18', 'person19', 'person2', 'person20', 'person21', 'person22', 'person23', 'person24', 'person25', 'person26', 'person27', 'person28', 'person29', 'person3', 'person4', 'person5', 'person6', 'person7', 'person8', 'person9', 'riding1', 'riding10', 'riding11', 'riding12', 'riding13', 'riding14', 'riding15', 'riding16', 'riding17', 'riding2', 'riding3',

In [ ]:
os.chdir('/content/DAC_SDC_2020_Yolov3_model/train/')


In [ ]:
!python3 train_yolo_v3.py

Streaming output truncated to the last 5000 lines.
[2020-09-22 04:18:48,541 train_yolo_v3.py] epoch [003] iter = 3860 loss = 0.008762 example/sec = 52.602 lr = 0.01000 
[2020-09-22 04:18:52,447 train_yolo_v3.py] epoch [003] iter = 3870 loss = 0.009209 example/sec = 51.122 lr = 0.01000 
[2020-09-22 04:18:56,424 train_yolo_v3.py] epoch [003] iter = 3880 loss = 0.009904 example/sec = 52.760 lr = 0.01000 
[2020-09-22 04:19:00,261 train_yolo_v3.py] epoch [003] iter = 3890 loss = 0.012048 example/sec = 50.366 lr = 0.01000 
[2020-09-22 04:19:04,127 train_yolo_v3.py] epoch [003] iter = 3900 loss = 0.008016 example/sec = 52.420 lr = 0.01000 
[2020-09-22 04:19:08,025 train_yolo_v3.py] epoch [003] iter = 3910 loss = 0.008073 example/sec = 48.229 lr = 0.01000 
[2020-09-22 04:19:11,953 train_yolo_v3.py] epoch [003] iter = 3920 loss = 0.010325 example/sec = 47.380 lr = 0.01000 
[2020-09-22 04:19:15,871 train_yolo_v3.py] epoch [003] iter = 3930 loss = 0.008241 example/sec = 52.196 lr = 0.01000 
[2020